# modular design

## separation of concerns

isolate different parts of a program that address different concerns

modular component can be developed and tested independently

In [3]:
def search(query, ranking=lambda r: -r.stars):
    results = [r for r in Restaurant.all if query in r.name]
    return sorted(results, key=ranking)

def reviewed_both(r, s):
    return fast_overlap(r.reviewers, s.reviewers)
    return len([x for x in r.reviewers if x in s.reviewers])

def fast_overlap(s, t):
    """Return the overlap between sorted S and sorted T.

    >>> fast_overlap([2, 3, 5, 6, 7], [1, 4, 5, 6, 7, 8])
    3
    """
    count, i, j = 0, 0, 0
    while i < len(s) and j < len(t):
        if s[i] == t[j]:
            count, i, j = count + 1, i + 1, j + 1
        elif s[i] < t[j]:
            i += 1
        else:
            j += 1
    return count

class Restaurant:
    all = []
    def __init__(self, name, stars, reviewers):
        self.name, self.stars = name, stars
        self.reviewers = sorted(reviewers)
        Restaurant.all.append(self)

    def similar(self, k, similarity=reviewed_both):
        """Return the K most similar restaurants to SELF."""
        others = list(Restaurant.all)
        others.remove(self)
        return sorted(others, key=lambda x: -similarity(self, x))[:k]

    def __repr__(self):
        return '<' + self.name + '>'

import json

reviewers_for_restaurant = {}
for line in open('reviews.json'):
    r = json.loads(line)
    biz = r['business_id']
    if biz not in reviewers_for_restaurant:
        reviewers_for_restaurant[biz] = [r['user_id']]
    else:
        reviewers_for_restaurant[biz].append(r['user_id'])

for line in open('restaurants.json'):
    r = json.loads(line)
    reviewers = reviewers_for_restaurant[r['business_id']]
    Restaurant(r['name'], r['stars'], reviewers)

results = search('Thai')
for r in results:
    print(r, 'is similar to', r.similar(3))

<Thai Basil Cuisine> is similar to [<Gypsy's Trattoria Italiano>, <Top Dog>, <Smart Alec's Intelligent Food>]
<Thai Noodle II> is similar to [<La Cascada Taqueria>, <Cafe Milano>, <Chinese Express>]
<Jasmine Thai> is similar to [<Hummingbird Cafe>, <La Burrita 2>, <The Stuffed Inn>]
<Berkeley Thai House> is similar to [<Smart Alec's Intelligent Food>, <Thai Basil Cuisine>, <Top Dog>]
<Viengvilay Thai Cuisine> is similar to [<La Val's Pizza>, <Thai Basil Cuisine>, <La Burrita 2>]
